In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense

In [5]:
df = pd.read_csv("../../dataset/swat-minimized.csv", delimiter=";", decimal=",")
# df = df[16000:]
df = df.drop("Normal/Attack", axis=1)
df = df.drop("Timestamp", axis=1)

In [6]:
features_dropped = ["AIT201", "AIT202", "AIT203", "P201", "AIT401",
"AIT402", "AIT501", "AIT502", 'AIT503', "AIT504", "FIT503", "FIT504",
"PIT501", "PIT502", "PIT503"]

df = df.drop(columns=features_dropped)

In [7]:
n = len(df)
train_df = df[0:int(n*0.8)]
test_df = df[int(n*0.8):]

In [8]:
print("Features used: ", df.columns)
print(len(df.columns))

Features used:  Index(['FIT101', 'LIT101', 'MV101', 'P101', 'P102', 'FIT201', 'MV201', 'P202',
       'P203', 'P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301',
       'MV302', 'MV303', 'MV304', 'P301', 'P302', 'FIT401', 'LIT401', 'P401',
       'P402', 'P403', 'P404', 'UV401', 'FIT501', 'FIT502', 'P501', 'P502',
       'FIT601', 'P601', 'P602', 'P603'],
      dtype='object')
36


In [27]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
	"""
	We create time windows to create X and y features.
	For example, if we choose a window of 30, we will create a dataset formed by 30 points as X
	"""
	dataset = tf.data.Dataset.from_tensor_slices(series)
	dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
	dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
	dataset = dataset.shuffle(shuffle_buffer)
	dataset = dataset.map(lambda window: (window[:-1], window[-1]))
	dataset = dataset.batch(batch_size).prefetch(1)
	return dataset

In [28]:
dataset = windowed_dataset(train_df, 5, 32, 1000)

In [39]:
train_df.to_numpy().shape

(799, 36)

In [46]:
def create_uncompiled_model():
  # define a sequential model
  model = tf.keras.models.Sequential([ 
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True, input_shape=(-1, 36))),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
      tf.keras.layers.Dense(36),
  ]) 

  return model

In [47]:
def create_model():
    tf.random.set_seed(51)
  
    model = create_uncompiled_model()
  
    model.compile(loss=tf.keras.losses.MeanSquaredError(), 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=["mae"])  
    return model

In [48]:
model = create_model()

# we train for 20 epochs with and assign the callback
history = model.fit(dataset, epochs=1)

25/25 [==============================] - 36s 632ms/step - loss: 1371.5447 - mae: 11.1690


In [51]:
def model_forecast(model, series, window_size):
    """This function converts the input series into a dataset with time windows for forecasting"""
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [64]:
test = df.to_numpy()
print(test[0])
pred = model.predict(test[0][np.newaxis][np.newaxis])

print(pred)

[0.000000e+00 1.243135e+02 1.000000e+00 1.000000e+00 1.000000e+00
 0.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00
 1.000000e+00 1.000000e+00 2.560983e+00 2.562210e-04 1.385061e+02
 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00 1.000000e+00
 1.000000e+00 0.000000e+00 1.338503e+02 1.000000e+00 1.000000e+00
 1.000000e+00 1.000000e+00 1.000000e+00 1.538067e-03 1.408992e-03
 1.000000e+00 1.000000e+00 2.563040e-04 1.000000e+00 1.000000e+00
 1.000000e+00]
1/1 [==============================] - 3s 3s/step
[[ 0.3976486   6.200303    0.9766389   0.72598594  0.5523159   0.11366516
   0.8682514   0.54315764  0.6672812   0.77033067  0.5518025   0.7041555
   1.8040266  -0.04833576  5.7031274   0.62101424  0.63455474  0.878651
   0.8848462   0.6672926   0.5831128   0.12936716  6.4631824   0.8976166
   0.3642763   0.7275253   0.53227836  0.9897749   0.04938983 -0.07585277
   0.8468361   0.73668593  0.03170721  0.66168594  0.8987491   0.93287766]]
